In [51]:
import requests
import time
import pandas as pd
import sqlite3
from io import BytesIO

def main():
    # --- 設定項目 ---
    # 対象のe-StatファイルダウンロードURL
    target_url = "https://www.e-stat.go.jp/stat-search/file-download?statInfId=000040209841&fileKind=0"
    # 保存するDBファイル名
    db_name = "estat_data.db"
    # サーバー負荷への配慮（秒）
    wait_time = 3
    # 連絡先情報（マナーとしてUser-Agentに記載）
    headers = {
        'User-Agent': 'ResearchBot (Contact: your-email@example.com)'
    }

    print(f"1. サーバー負荷に配慮し、{wait_time}秒待機します...")
    time.sleep(wait_time)

    try:
        # 2. ファイルのダウンロード
        print("2. データを取得中...")
        response = requests.get(target_url, headers=headers, timeout=30)
        response.raise_for_status()

        # 3. Excelとして読み込み
        # Excelファイル（.xlsx）の実体はZIP構造のため、BytesIOで展開して読み込みます
        print("3. Excelファイルを解析中（全シート対象）...")
        with BytesIO(response.content) as excel_file:
            # sheet_name=None を指定すると、全シートを辞書形式 {シート名: DataFrame} で読み込みます
            # 統計表のヘッダーが複雑な場合は、必要に応じて skiprows を追加してください
            dict_df = pd.read_excel(excel_file, sheet_name=None)

        # 4. SQLiteデータベースへ保存
        print(f"4. データベース '{db_name}' へ保存を開始します...")
        with sqlite3.connect(db_name) as conn:
            for sheet_name, df in dict_df.items():
                # テーブル名に不適切な文字（スペースやハイフン）をアンダースコアに置換
                table_name = sheet_name.replace(' ', '_').replace('-', '_').replace('.', '_')
                
                # DBへ書き出し
                # if_exists='replace': 既にテーブルがあれば作り直す
                df.to_sql(table_name, conn, if_exists='replace', index=False)
                print(f"   [成功] シート '{sheet_name}' -> テーブル '{table_name}' ({len(df)}件)")

        print("\n--- すべての工程が正常に完了しました ---")

    except requests.exceptions.RequestException as e:
        print(f"\n[通信エラー] ダウンロードに失敗しました: {e}")
    except Exception as e:
        print(f"\n[エラー] 予期せぬ問題が発生しました: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

1. サーバー負荷に配慮し、3秒待機します...
2. データを取得中...
3. Excelファイルを解析中（全シート対象）...
4. データベース 'estat_data.db' へ保存を開始します...
   [成功] シート 'e001_1' -> テーブル 'e001_1' (77件)

--- すべての工程が正常に完了しました ---
